## Problem 6.3: Analysis of FRAP data (40 pts)¶

Attribution: Zhiyang and Xinhong did the coding, the whole group discussed together about the models and methods.

In [ ]:
import numpy as np
import pandas as pd
import scipy.special
import scipy.stats as st
import statsmodels.tools.numdiff as smnd

import bebi103

import altair as alt
import bokeh.io
bokeh.io.output_notebook()

The model given by the problem set:

\begin{align}
I_\mathrm{norm}(t) \equiv I(t)/I_0 &= 
f_f\left(1 - f_b\,\frac{4 \mathrm{e}^{-k_\mathrm{off}t}}{d_x d_y}\,\psi_x(t)\,\psi_y(t)\right),\\[1mm]
\text{where } \psi_i(t) &= \frac{d_i}{2}\,\mathrm{erf}\left(\frac{d_i}{\sqrt{4Dt}}\right)
-\sqrt{\frac{D t}{\pi}}\left(1 - \mathrm{e}^{-d_i^2/4Dt}\right),
\end{align}

We will first turn the equations of the model into functions, where we want to compute the $I_{norm}(t)$ first.

In [ ]:
def fluo_bleach_psi(dx, D, t):
    '''Specifies psi in Inorm equation.'''
    return dx / 2 * scipy.special.erf(dx / np.sqrt(4 * D * t)) - \
           np.sqrt(D * t / np.pi) * (1 - np.exp(-np.power(dx,2) / (4 * D * t)))

def fluo_bleach_cal(t, f_b, f_f, k_off, D, dx=40*0.138, dy=40*0.138):
    '''Piecewise equation for Inorm, where before time 0, Inorm=1.'''
    # Before photobleaching happens the normalized intensity is 1
    if t < 0:
        return 1
    # When photobleaching happens, take the limit of t approaching 0
    elif t == 0:
        return f_f * (1 - f_b)
    # Other time points can be computed as shown above
    else:
        psi_x = fluo_bleach_psi(dx, D, t)
        psi_y = fluo_bleach_psi(dy, D, t)
        return f_f * (1 - f_b * 4 * np.exp(-k_off * t) / (dx * dy) * psi_x * psi_y)

Let's read in the data and take a look.

In [ ]:
# read in the dataset
df = pd.read_csv('../data/hw_4.1_frap_image_processing_results.csv')
df.head()

First let's plot the data and get an idea of how it looks.

In [ ]:
# Plot all the trials
alt.Chart(df
    ).mark_line(
        strokeJoin='bevel'
    ).encode(
        x=alt.X('time:Q', title='time (s)'),
        y=alt.Y('normalized_intensity:Q', title='normalized intensity'),
        color='trial:N',
        order='time:Q'
    )

Note here we use the normalized intensity which is normalized by the mean of the intensity before the photobleaching for each trial respectively. We think it is better to use this to compare among trials.

Even though this may cause problems like neglected $I_0$ in the model, we assume the reason to have another parameter $I_0$ is to account for the measurement noise, which could also be done by using a normal distribution with mean of $I_{norm}(t, f_b, f_f, k_{off}, D)$ and standard deviation of some $\sigma$. This is actually equivalent to having a normal distribution $I_0$ with the mean of mean intensity by time 0 for each trial and some standard deviation. The only problem is that $\sigma$ could be different among trials, but we think it is appropriate to assume similar measurement noise here.

After looking at the data and consider the definition of parameters, we hypothesize the distribution of different parameters:

We think the data $I(t)$ in general would follow a normal distribution where the mean is $I_{norm}(t, f_b, f_f, k_{off}, D)$ and the standard deviation is $\sigma$. 

* $f_b$ is the fraction of bleached fluorophores, it has the bound of 0~1, and it shold be close to 1, so we speculate it would follow a beta distribution.  
* $f_f$ is the fraction of total fluorescent species left after photobleaching, it would be similar to fb, should also follow beta distribution.  
* $k_{off}$ is chemical rate constant, the situation is similar to D, so it should also lies in a lognormal distribution.  
* $D$ is the diffusion coefficent, we assume it could be similar across trials, which indicates  a sharp peak, also it has a lower bound (0), but not an upper bound. So lognormal distribution would best to describe D.  
* $\sigma$ accounts for the measurement noise which usually follows a half normal distribution. 


For $f_b$ and $f_f$, we basically look at the shape of the beta distribution and get the parameters we think is good to describe the situation here. For $k_{off}$ and $D$, we have no idea about what they should be like, so we choose a heavily tailed log normal distribution with some means we guess from the data. we set the standard deviation to 0.02 since the data is actually pretty clean.  

After getting the general idea of distribution of each parameters, let's specify each parameter:

In [ ]:
# Specify the distributions for each parameters in the model
def data_prior_pred(t, n_ppc_samples=1):
    '''
    Samples parameter values according to the prior and generates
    data y at the values given in x.
    '''
    sigma = np.abs(np.random.normal(0, 0.02, size=n_ppc_samples))
    k_off = np.random.lognormal(np.log(0.2), 0.5, size=n_ppc_samples)
    D = np.random.lognormal(np.log(0.1), 0.75, size=n_ppc_samples)
    fb = np.random.beta(6, 3, size=n_ppc_samples)
    ff = np.random.beta(15, 2, size=n_ppc_samples)
    
    data = np.array([np.random.normal(fluo_bleach_cal(t0, fb, ff, k_off, D),
                                      sigma, size=1) 
                        for t0 in t])
    return np.concatenate(data)

In summary, our prior is:

$$f_b\sim Beta(6,3)$$
$$f_f\sim Beta(15,2)$$
$$k_{off}\sim LogNorm(\ln(0.2), 0.5)$$
$$D\sim LogNorm(\ln (0.1), 0.75)$$
$$\sigma \sim Norm(0, 0.01)$$


And the likelihood is:

$$I(t) \sim Norm(I_{norm}(t, f_b, f_f, k_{off}, D), \sigma)$$ 

Let's use Trial0 to do a prior predictive check! First let's slice out the data for trial0.

In [ ]:
# Slice out the data belongs to trial0
t0 = df.loc[df['trial']==0, 'time'].values
v0 = df.loc[df['trial']==0, 'normalized_intensity'].values

Now we can plot several trials of the simulated data using the prior predictive check against the real data.

In [ ]:
# prior check use the data in trial0
colors = bokeh.palettes.all_palettes['Category20'][20]

p = bokeh.plotting.figure(height=300, width=450,
                          x_axis_label='Time(s)',
                          y_axis_label='Normalized intensity')

# Plot simulated data
for i in range(20):
    p.line(t0, data_prior_pred(t0), color=colors[i],
             alpha=0.8)

# Plot original data
p.circle(t0, v0, color='black', size=4)
bokeh.io.show(p)

The black dots are the original data we acquired from the dataset, the colored lines are the simulated data generated from our prior. We could see that our prior cover most of the real data, which indicates our prior seems reasonable. However, since we are going to use this same prior for all the trials, it is possible that they didn't fit with the data in other trial very well, so when we do prior check for each trial, we may have to go back to tune the parameters of our model. But let's move on to following analysis of Trial0 with this prior now.

Let's specify the log-likelihood and log-prior.

In [ ]:
# Specify the log_prior
def log_prior(params):
    
    if (params < 0).any():
        return -np.inf
    
    fb, ff, k_off, D, sigma = params

    log_prior = st.beta.logpdf(fb, 6, 3)
    log_prior += st.beta.logpdf(ff, 15, 1)
    log_prior += st.lognorm.logpdf(k_off, 0.5, loc=0, scale=0.2)
    log_prior += st.lognorm.logpdf(D, 0.75, loc=0, scale=0.3)
    log_prior += st.halfnorm.logpdf(sigma, 0, 0.01)
    
    return log_prior

# Specify the log_likelihood
def log_like(params, t, y):
    fb, ff, k_off, D, sigma = params
    
    log_like = np.sum([st.norm.logpdf(y0, fluo_bleach_cal(t0, fb, ff, k_off, D), sigma)
                       for y0, t0 in zip(y, t)])
    
    return log_like

We would also like to calculate the log-posterior.

In [ ]:
# Calculate the posterior using the prior and likelihood
def log_post(params, t, y):
    return log_prior(params) + log_like(params, t, y)

def neg_log_post(params, t, y):
    return -log_post(params, t, y)

Since we have 5 parameters in total, it's hard to plot the posterior. So we will directly go ahead and find the MAP values.  

After several trials of finding MAP values using different methods, we find that 'SLSQP' works the best for us. We are not quite sure about why this method works but not the others. For instance, when we use 'Powell' method Justin used in the Tutorial, it gives us some weird number larger than 200.

Let's compute the MAP values.

In [ ]:
# Specify arguments
args_0 = (t0, v0)

# Choose initial conditiona
params_0 = [0.9, 0.9, 0.2, 0.3, 0.01]

# Compute the MAP values
opt_res_0 = scipy.optimize.minimize(neg_log_post, params_0, args=args_0,
                                    method='SLSQP')

And take a look at the values.

In [ ]:
# Take a look at the Map values 
popt_0 = opt_res_0.x
print(popt_0)

These parameter values look reasonable based on the definition of parameters and our intuition for their distribution. So let's plug these paramters into our model and plot the simulated line using the data sample from this generative model. We can also plot the measured data in the same plot to compare the difference.

In [ ]:
# Do a posterior check with the real data

# Plug in the paramters we get and generate some simulated data
def data_prior_pred_0(t, n_ppc_samples=1):
    fb, ff, k_off, D, sigma = popt_0
    data = np.array([np.random.normal(fluo_bleach_cal(t0, fb, ff, k_off, D),
                                      sigma, size=1) 
                        for t0 in t])
    return np.concatenate(data)


colors = bokeh.palettes.all_palettes['Category20'][20]

p = bokeh.plotting.figure(height=300, width=450,
                          x_axis_label='Time(s)',
                          y_axis_label='Normalized intensity')

# Plot simulated data for 20 'simulated trials'
for i in range(20):
    p.line(t0, data_prior_pred_0(t0), color=colors[i],
             alpha=0.8)

# Plot original data
p.circle(t0, v0, color='black', size=4)
bokeh.io.show(p)

The black dots are the original data we acquired from the dataset, the colored lines are several trials of simulated data generated from our posterior. We could see they fit with each other quite well, indicating that our estimation of paramters are quite reasonable. So let's move on and calculate the 95% credible region for these parameters.

In [ ]:
hes = smnd.approx_hess(popt_0, log_post, args=args_0)

# Compute the covariance matrix
cov = -np.linalg.inv(hes)

# Look at it
for i in range(5):
    print(abs(cov[i][i]))
    

Let's put these values into a nice form so we can read the credible regions conveniently.

In [ ]:
# For convenience...
fb_MAP_0, ff_MAP_0, k_off_MAP_0, D_MAP_0, sigma_MAP_0  = popt_0

# Print results
print("""
Most probable parameters for Trial0
-----------------------------------
fb_MAP_0 = {0:.2f} ± {1:.2f} 
ff_MAP_0 = {2:.3f} ± {3:.3f}
k_off_MAP_0 = {4:.3f} ± {5:.3f}
D_MAP_0 = {6:.3f} ± {7:.3f}
sigma_MAP_0 = {8:.3f} ± {9:.3f}
""".format(fb_MAP_0, 2* np.sqrt(abs(cov[0, 0])), 
           ff_MAP_0, 2* np.sqrt(abs(cov[1, 1])), 
           k_off_MAP_0, 2* np.sqrt(abs(cov[2, 2])),
           D_MAP_0, 2* np.sqrt(abs(cov[3, 3])),
           sigma_MAP_0, 2* np.sqrt(abs(cov[4, 4]))))

We get the values of the parameters and their 95% credible region! They looks fairy reasonable. Let's do the simailar analysis on all the other trials.   

To perform the similar method on all the trial, we would like to summarize the function into several blocks, and call the block for each trial to perform all the operation.

First we write a function to perform and plot the prior predictive check.

In [ ]:
def slice_data_prior_check(i, N_iter=20):
    '''Performs 20 simulations from the prior and plots
    the simulated data against the actual data for trial i.'''
    ti = df.loc[df['trial']==i, 'time'].values
    vi = df.loc[df['trial']==i, 'normalized_intensity'].values
    
    colors = bokeh.palettes.all_palettes['Category20'][20]

    p1 = bokeh.plotting.figure(height=300, width=450,
                              x_axis_label='time(s)',
                              y_axis_label='intensity',
                              title='prior')

    # Plot simulated data
    for i in range(N_iter):
        p1.line(ti, data_prior_pred(ti), color=colors[i], alpha=0.8)

    # Plot original data
    p1.circle(ti, vi, color='black', size=4)
      
    return ti, vi, p1

Now we write a function to compute MAP values.

In [ ]:
def compute_Map_values(i, ti, vi, params_i):
    '''Computes MAP values using SLSQP method.'''
    
    # Choose initial condition
    params = params_i

    # Compute the MAP values
    opt_res_i = scipy.optimize.minimize(neg_log_post, 
                                        params, 
                                        args=(ti, vi),
                                        method='SLSQP')
    #return Map_values
    return opt_res_i.x

Let's write functions to plot the simulated line based on the MAP parameters and original data.

In [ ]:
def data_prior_pred_back(ti, popt_i, n_ppc_samples=1):
    '''Predictive check using MAP values'''
    fb, ff, k_off, D, sigma = popt_i
    data = np.array([np.random.normal(fluo_bleach_cal(t, fb, ff, k_off, D),
                                      sigma, 
                                      size=1) 
                     for t in ti])
    
    return np.concatenate(data)

def plot_best_fit_line(i, ti, vi, popt_i, N_iter=20):
    '''Plots simulated data against original data.'''
    colors = bokeh.palettes.all_palettes['Category20'][20]

    p2 = bokeh.plotting.figure(height=300, width=450,
                              x_axis_label='time(s)',
                              y_axis_label='intensity',
                              title = 'MAP predictive check')

    # Plot simulated data
    for i in range(N_iter):
        p2.line(ti, data_prior_pred_back(ti, popt_i), color=colors[i],
               alpha=0.8)

    # Plot original data
    p2.circle(ti, vi, color='black', size=4)
    
    return p2
    

Now we write a function to compute the 95% credible regions for the parameters.

In [ ]:
def present_parameter_with_95(i, popt_i, args_i):
    '''Calculates the 95% credible regions for the parameters and prints them
    in a formatted way.'''
    # Using very small step size due to some values close to upper bound.
    hes = smnd.approx_hess(popt_i, log_post, args=args_i,  epsilon=1e-8)
    
    # Compute the covariance matrix
    cov = -np.linalg.inv(hes)

    # Print results
    print("Trial", i)
    print("""
    Most probable parameters 
    ------------------------
    fb_MAP = {0:.2f} ± {1:.2f} 
    ff_MAP = {2:.3f} ± {3:.3f}
    k_off_MAP = {4:.3f} ± {5:.3f}
    D_MAP = {6:.3f} ± {7:.3f}
    sigma_MAP = {8:.3f} ± {9:.3f}
    
    --------------------------------------------
    """.format(popt_i[0], 2* np.sqrt(abs(cov[0, 0])), 
               popt_i[1], 2* np.sqrt(abs(cov[1, 1])), 
               popt_i[2], 2* np.sqrt(abs(cov[2, 2])),
               popt_i[3], 2* np.sqrt(abs(cov[3, 3])),
               popt_i[4], 2* np.sqrt(abs(cov[4, 4]))))

    #return fb, ff, k_off, D, sigma
    return [popt_i[0], 2* np.sqrt(abs(cov[0, 0]))], [popt_i[1], 2* np.sqrt(abs(cov[1, 1]))], \
            [popt_i[2], 2* np.sqrt(abs(cov[2, 2]))], [popt_i[3], 2* np.sqrt(abs(cov[3, 3]))], \
            [popt_i[4], 2* np.sqrt(abs(cov[4, 4]))] 

After building the function blocks we need for the analysis, since we already performed analysis on Trial0, let's test the function with Trial0 to make sure everything is good. 

In [ ]:
# Run the function blocks on Trial0

# Perform prior predictive check
ti, vi, p1 = slice_data_prior_check(i)

# Compute MAP values
popt_i = compute_Map_values(i, ti, vi, [0.9, 0.9, 0.2, 0.3, 0.01])

# Plot MAP predictive check against data
args_i = (ti, vi)
p2 = plot_best_fit_line(i, ti, vi, popt_i)

# Compute 95% credible regions
result_i = present_parameter_with_95(i, popt_i, args_i)
fb = result_i[0][:]
ff = result_i[1][:]
k_off = result_i[2][:]
D = result_i[3][:]
sigma = result_i[4][:]

# Display plots
bokeh.io.show(bokeh.layouts.gridplot([p1, p2], ncols=2))

The black dots in both graphs are the measured data. Left graphs plot the simulated data from prior, right graphs plot simulated data from posterior. The result is the same as what we get from direct operation on trial0, let's move on and use this on other trials.

In [ ]:
# Run the function blocks on Trial0

# Perform prior predictive check
ti, vi, p1 = slice_data_prior_check(i)

# Compute MAP values
popt_i = compute_Map_values(i, ti, vi, [0.9, 0.9, 0.2, 0.3, 0.01])

# Plot MAP predictive check against data
args_i = (ti, vi)
p2 = plot_best_fit_line(i, ti, vi, popt_i)

# Compute 95% credible regions
result_i = present_parameter_with_95(i, popt_i, args_i)
fb = result_i[0][:]
ff = result_i[1][:]
k_off = result_i[2][:]
D = result_i[3][:]
sigma = result_i[4][:]

# Display plots
bokeh.io.show(bokeh.layouts.gridplot([p1, p2], ncols=2))

# For the rest
for i in range(7):
    # Perform prior predictive check
    ti, vi, p1 = slice_data_prior_check(i+1)
    
    # Compute MAP values
    popt_i = compute_Map_values(i+1, ti, vi, [0.9, 0.9, 0.2, 0.3, 0.01])
    
    # Plot MAP predictive check against data
    args_i = (ti, vi)
    p2 = plot_best_fit_line(i+1, ti, vi, popt_i)
    
    # Compute 95% credible regions
    result_i = present_parameter_with_95(i+1, popt_i, args_i)
    fb = np.vstack([fb, result_i[0][:]])
    ff = np.vstack([ff, result_i[1][:]])
    k_off = np.vstack([k_off, result_i[2][:]])
    D = np.vstack([D, result_i[3][:]])
    sigma = np.vstack([sigma, result_i[4][:]])
    
    # Display plots
    bokeh.io.show(bokeh.layouts.gridplot([p1, p2], ncols=2))

There is an issue in Trial2 where the MAP values for $f_f$ is too close to 1 so that we cannot have an accurate estimate of upper bound of 95% credible region. We change the step of Hes matrix to be very small but there is still something wrong in the result of Trial2. 

After running through each trial, we would like to look at all the parameters and plot the jitter plot for all the parameters with 95% credible regions.

Let's start with $f_b$.

In [ ]:
# Take a look at fb
print("fb[value, 95% credible region]:", '\n', fb)

We want to first put the estimate for $f_b$ in every trial into a dataframe for ease.

In [ ]:
# Put the estimate for fb in every trial into a dataframe
fb_estimate = pd.DataFrame(data={'value': fb[:,0], '95%': fb[:,1]})
fb_estimate.insert(0, 'trial', range(0, 0 + len(fb_estimate)))

# Take a look
fb_estimate

Now we will plot the estimate of fb with 95% credible regions to get a better sense of the parameter.

In [ ]:
# Slice out data for convenience
xs = fb_estimate['trial']
ys = fb_estimate['value']
yerrs = fb_estimate['95%']

# Plot the points
p_fb = bokeh.plotting.figure(height=300, width=450,
                              x_axis_label='trial',
                              y_axis_label='value',
                              title = 'estimate of fb with 95% credible regions')

p_fb.circle(xs, ys, color='green', size=5, line_alpha=0)


# create the coordinates for the errorbars
err_xs = []
err_ys = []

for x, y, yerr in zip(xs, ys, yerrs):
    err_xs.append((x, x))
    err_ys.append((y - yerr, y + yerr))

# plot them
p_fb.multi_line(err_xs, err_ys, color='green')

bokeh.plotting.show(p_fb)

Let's do the similar plot on the other parameters.

Plot the estimate of $f_f$ with 95% credible regions:

In [ ]:
# Put the estimate for ff in every trial into a dataframe
ff_estimate = pd.DataFrame(data={'value': ff[:,0], '95%': ff[:,1]})
ff_estimate.insert(0, 'trial', range(0, 0 + len(ff_estimate)))
ff_estimate

# for convenience..
xs = ff_estimate['trial']
ys = ff_estimate['value']
yerss = ff_estimate['95%']

# plot the points
p_ff = bokeh.plotting.figure(height=300, width=450,
                              x_axis_label='trial',
                              y_axis_label='value',
                              title = 'estimate of ff with 95% credible regions')

p_ff.circle(xs, ys, color='green', size=5, line_alpha=0)


# create the coordinates for the errorbars
err_xs = []
err_ys = []

for x, y, yerr in zip(xs, ys, yerrs):
    err_xs.append((x, x))
    err_ys.append((y - yerr, y + yerr))

# plot them
p_ff.multi_line(err_xs, err_ys, color='green')

bokeh.plotting.show(p_ff)

Plot the estimate of $k_{off}$ with 95% credible regions:

In [ ]:
#Plot the estimate of k_off with 95% credible region

# Put the estimate for k_off in every trial into a dataframe
k_off_estimate = pd.DataFrame(data={'value': k_off[:,0], '95%': k_off[:,1]})
k_off_estimate.insert(0, 'trial', range(0, 0 + len(k_off_estimate)))
k_off_estimate

# for convenience..
xs = k_off_estimate['trial']
ys = k_off_estimate['value']
yerss = k_off_estimate['95%']

# plot the points
p_k_off = bokeh.plotting.figure(height=300, width=450,
                              x_axis_label='trial',
                              y_axis_label='value',
                              title = 'estimate of k_off with 95% credible regions')

p_k_off.circle(xs, ys, color='blue', size=5, line_alpha=0)


# create the coordinates for the errorbars
err_xs = []
err_ys = []

for x, y, yerr in zip(xs, ys, yerrs):
    err_xs.append((x, x))
    err_ys.append((y - yerr, y + yerr))

# plot them
p_k_off.multi_line(err_xs, err_ys, color='blue')

bokeh.plotting.show(p_k_off)

Something seems wrong in the Trial2, we proceed and will get back to it. Plot the estimate of D with 95% credible region:

In [ ]:
#Plot the estimate of D with 95% credible region

# Put the estimate for D in every trial into a dataframe
D_estimate = pd.DataFrame(data={'value': D[:,0], '95%': D[:,1]})
D_estimate.insert(0, 'trial', range(0, 0 + len(D_estimate)))
D_estimate

# for convenience..
xs = D_estimate['trial']
ys = D_estimate['value']
yerss = D_estimate['95%']

# plot the points
p_D = bokeh.plotting.figure(height=300, width=450,
                              x_axis_label='trial',
                              y_axis_label='value',
                              title = 'estimate of D with 95% credible region')

p_D.circle(xs, ys, color='orange', size= 5, line_alpha=0)


# create the coordinates for the errorbars
err_xs = []
err_ys = []

for x, y, yerr in zip(xs, ys, yerrs):
    err_xs.append((x, x))
    err_ys.append((y - yerr, y + yerr))

# plot them
p_D.multi_line(err_xs, err_ys, color='black')

bokeh.plotting.show(p_D)

The error bar in D plot looks very small (the little black lines) because of the scale, but we can still see that the Trial2 is kind of strange.

Plot the estimate of sigma with 95% credible region:

In [ ]:
#Plot the estimate of sigma with 95% credible region

# Put the estimate for sigma in every trial into a dataframe
sigma_estimate = pd.DataFrame(data={'value': sigma[:,0], '95%': sigma[:,1]})
sigma_estimate.insert(0, 'trial', range(0, 0 + len(sigma_estimate)))
sigma_estimate

# for convenience..
xs = sigma_estimate['trial']
ys = sigma_estimate['value']
yerss = sigma_estimate['95%']

# plot the points
p_sigma = bokeh.plotting.figure(height=300, width=450,
                              x_axis_label='trial',
                              y_axis_label='value',
                              title = 'estimate of sigma with 95% credible region')

p_sigma.circle(xs, ys, color='purple', size=5, line_alpha=0)


# create the coordinates for the errorbars
err_xs = []
err_ys = []

for x, y, yerr in zip(xs, ys, yerrs):
    err_xs.append((x, x))
    err_ys.append((y - yerr, y + yerr))

# plot them
p_sigma.multi_line(err_xs, err_ys, color='purple')

bokeh.plotting.show(p_sigma)

We can also visualize all the parameters together by trial with their 95% confidence intervals.

In [ ]:
bokeh.io.show(bokeh.layouts.gridplot([p_fb, p_ff, p_k_off, p_D, p_sigma], ncols=2))

We can see that there is something different in Trial2 from others. We are not sure why this is happening, possbily due to the very high estimate of $f_f$ but other parameters are also very different from those in other trials, which should be similar. It might be because of the MAP optimizer or coding of the posterior, but we did not find anything wrong in the function. We would like to just put Trial2 aside if we will further analyze the data.

In [ ]:
%load_ext watermark

In [ ]:
%watermark -v -p numpy,pandas,statsmodels,altair,scipy,bokeh,jupyterlab

# Grading

**Grade: 36.5/40**

Very nice work overall!  Clearly explanations and work flow.

Some comments:
- Just because you use normalized data doesn’t mean that you can ignore $I_0$ in your model.  Your $I_0$ would simply have a change of units such that a reasonable prediction for it would be approximately 1.  You even mention that you should have another parameter $I_0$ to account for measurement noise, but then don’t make a prior on it. (-2 pts)
- DO NOT PLOT DATA WITH PRIOR PREDICTIVE CHECKS.  It’s not about whether the data match the prior predictive checks, but rather whether the prior predictive checks on their own give something reasonable that you would expect from your model. (-1 pt)
- In your PPC plot, it is unclear what the smoother colored lines are compared to the noisier ones.
- 95% confidence intervals are technically 1.96 x sqrt(cov), not 2 times.  Also you SHOULD NOT take absolute value of covariance matrix...  Diagonal values should always be positive! (-0.5 pt)
- it is interesting that you saw Trial 2 giving outliers for parameter estimates, where we observed Trial 7 being an outlier.  Could just be due to the priors you set up, or I just didn’t catch something…
